In [3]:
from PIL import Image
import numpy as np
from keras.models import load_model
#img = Image.open("F:\\data\\validation\\R_TO_L\\human13_0_2862.jpeg")
count_LR = 0
count_RL = 0
for i in range(1,10):
    img = Image.open("F:\\test_crawl\\Tarun_crawl_"+str(i)+".png")
    rgb_im = img.convert('RGB')
    print rgb_im.size
    img_width, img_height = 150, 150
    imgvalue = rgb_im.resize((img_width,img_height))
    imgvalue = np.array(imgvalue.getdata()).reshape(imgvalue.size[0], imgvalue.size[1], 3)
#imgvalue = imgvalue.convert(mode='F')
    imgvalue =  imgvalue.astype('float32')
    imgNew = imgvalue/255
#imgNew = imgNew.transpose((2,0,1))
    imgval = np.expand_dims(imgNew,axis=0)      
    print imgval.shape                    
    numpy_img = np.array(imgval)
#print numpy_img
    model = load_model('F:\\data\\updated_model_fine_tuned_crawl_vs_animal.h5')
    model.load_weights('F:\\data\\updated_weights_fine_tuned_crawl_vs_animal.h5')
    a = model.predict(numpy_img)
    print a
    if(a[0][0] > a[0][1]):
        print "animal"
        count_LR = count_LR + 1
    elif(a[0][1] > a[0][0]):
        print "crawl"
        count_RL = count_RL + 1
    else:
        print "No orientation detected"
print "animal"+str(count_LR)
print "crawl"+str(count_RL) 

(640, 480)
(1L, 150L, 150L, 3L)
[[  1.00000000e+00   1.81185036e-23]]
animal
(640, 480)
(1L, 150L, 150L, 3L)
[[  1.00000000e+00   9.67398127e-22]]
animal
(640, 480)
(1L, 150L, 150L, 3L)
[[  1.00000000e+00   2.88907899e-19]]
animal
(640, 480)
(1L, 150L, 150L, 3L)
[[  1.00000000e+00   2.29475704e-16]]
animal
(640, 480)
(1L, 150L, 150L, 3L)
[[  1.00000000e+00   6.78981611e-16]]
animal
(640, 480)
(1L, 150L, 150L, 3L)
[[  1.00000000e+00   8.20261545e-17]]
animal
(640, 480)
(1L, 150L, 150L, 3L)
[[  1.00000000e+00   1.54240119e-18]]
animal
(640, 480)
(1L, 150L, 150L, 3L)
[[  1.00000000e+00   3.00930980e-16]]
animal
(640, 480)
(1L, 150L, 150L, 3L)
[[  1.00000000e+00   5.84231108e-09]]
animal
animal9
crawl0


In [19]:
############################################analysisng crawl_data vs animal#################

'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'F:\\data_crawl\\train'
validation_data_dir = 'F:\\data_crawl\\validation'
nb_train_samples = 1408
nb_validation_samples = 329
epochs = 10
batch_size = 50

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255,horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('F:\\data\\updated_weights_fine_tuned_crawl_vs_animal.h5')
model.save('F:\\data\\updated_model_fine_tuned_crawl_vs_animal.h5')

Found 1325 images belonging to 2 classes.
Found 330 images belonging to 2 classes.
Epoch 1/10
 4/28 [===>..........................] - ETA: 181s - loss: 2.6710 - acc: 0.4950

KeyboardInterrupt: 

In [11]:
from PIL import Image
import numpy as np
from keras.models import load_model
#img = Image.open("F:\\data\\validation\\R_TO_L\\human13_0_2862.jpeg")
count_LR = 0
count_RL = 0
for i in range(1,10):
    img = Image.open("F:\\test_crawl\\Tarun_crawl_"+str(i)+".png")
    rgb_im = img.convert('RGB')
    print rgb_im.size
    img_width, img_height = 150, 150
    imgvalue = rgb_im.resize((img_width,img_height))
    imgvalue = np.array(imgvalue.getdata()).reshape(imgvalue.size[0], imgvalue.size[1], 3)
#imgvalue = imgvalue.convert(mode='F')
    imgvalue =  imgvalue.astype('float32')
    imgNew = imgvalue/255
#imgNew = imgNew.transpose((2,0,1))
    imgval = np.expand_dims(imgNew,axis=0)      
    print imgval.shape                    
    numpy_img = np.array(imgval)
#print numpy_img
    #model = load_model('F:\\data\\updated_model_fine_tuned_crawl_vs_animal.h5')
    model.load_weights('F:\\data\\updated_weights_fine_tuned_crawl_vs_animal.h5')
    a = model.predict(numpy_img)
    print a
    if(a[0][0] > a[0][1]):
        print "animal"
        count_LR = count_LR + 1
    elif(a[0][1] > a[0][0]):
        print "crawl"
        count_RL = count_RL + 1
    else:
        print "No orientation detected"
print "animal"+str(count_LR)
print "crawl"+str(count_RL) 

(640, 480)
(1L, 150L, 150L, 3L)
[[  9.99993503e-01   6.52652398e-06]]
animal
(640, 480)
(1L, 150L, 150L, 3L)
[[  9.99987543e-01   1.24678345e-05]]
animal
(640, 480)
(1L, 150L, 150L, 3L)
[[  9.99980390e-01   1.95890752e-05]]
animal
(640, 480)
(1L, 150L, 150L, 3L)
[[  9.99973834e-01   2.61673249e-05]]
animal
(640, 480)
(1L, 150L, 150L, 3L)
[[  9.99966800e-01   3.32179661e-05]]
animal
(640, 480)
(1L, 150L, 150L, 3L)
[[  9.99978244e-01   2.17310899e-05]]
animal
(640, 480)
(1L, 150L, 150L, 3L)
[[  9.99988854e-01   1.11543659e-05]]
animal
(640, 480)
(1L, 150L, 150L, 3L)
[[  9.99805212e-01   1.94775246e-04]]
animal
(640, 480)
(1L, 150L, 150L, 3L)
[[  9.99823928e-01   1.76044952e-04]]
animal
animal9
crawl0


In [3]:
from PIL import Image
import numpy as np
#img = Image.open("F:\\data\\validation\\R_TO_L\\human13_0_2862.jpeg")
img = Image.open("F:\\img_RGB\\vlcsnap-2017-06-19-15h16m16s198.png")
rgb_im = img.convert('RGB')
print rgb_im.size
img_width, img_height = 150, 150
imgvalue = rgb_im.resize((img_width,img_height))
imgvalue = np.array(imgvalue.getdata()).reshape(imgvalue.size[0], imgvalue.size[1], 3)
#imgvalue = imgvalue.convert(mode='F')
imgvalue =  imgvalue.astype('float32')
imgNew = imgvalue/255
#imgNew = imgNew.transpose((2,0,1))
imgval = np.expand_dims(imgNew,axis=0)      
print imgval.shape                    
numpy_img = np.array(imgval)
#print numpy_img
model.load_weights('F:\\data\\first_try.h5')
model.predict(numpy_img)

(640, 480)
(1L, 150L, 150L, 3L)


NameError: name 'model' is not defined

In [2]:
from PIL import Image
import numpy as np
#img = Image.open("F:\\data\\validation\\R_TO_L\\human13_0_2862.jpeg")
count_LR = 0
count_RL = 0
for i in range(1,6):
    img = Image.open("F:\\img_RGB\\man_"+str(i)+".png")
    rgb_im = img.convert('RGB')
    print rgb_im.size
    img_width, img_height = 150, 150
    imgvalue = rgb_im.resize((img_width,img_height))
    imgvalue = np.array(imgvalue.getdata()).reshape(imgvalue.size[0], imgvalue.size[1], 3)
#imgvalue = imgvalue.convert(mode='F')
    imgvalue =  imgvalue.astype('float32')
    imgNew = imgvalue/255
#imgNew = imgNew.transpose((2,0,1))
    imgval = np.expand_dims(imgNew,axis=0)      
    print imgval.shape                    
    numpy_img = np.array(imgval)
#print numpy_img
    model.load_weights('F:\\data\\first_try.h5')
    a = model.predict(numpy_img)
    if(a[0][0] > a[0][1]):
        print "LEFT to Right Orientation"
        count_LR = count_LR + 1
    if(a[0][1] > a[0][0]):
        print "Right to Left Orientation"
        count_RL = count_RL + 1
    else:
        print "No orientation detected"
print "LR detection"+str(count_LR)
print "RL detection"+str(count_RL) 

(640, 480)
(1L, 150L, 150L, 3L)


NameError: name 'model' is not defined

In [21]:
#################################GETTING AUGMENTED DATA for human

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy
import os
datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='reflect')
root = "F:\\"
directory = root + "Female"
count = 0
for file in os.listdir(directory):
    #print file
    file_name = os.path.join(directory, file)
    img = load_img(file_name)  
    # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    #x = reshape(x,())
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
    #x = numpy.flipud(x)
    # the .flow() command below generates batches of randomly transformed images
    # and saves the results to the `preview/` directory
    i = 0
    #x = numpy.rot90(x,)
    #x = numpy.fliplr(x)
    #flip_p=0.5
    #flip_selector = numpy.random.binomial(1, flip_p, size=(x.shape[0]) == 1)
    #x[flip_selector,:,:,:] = x[flip_selector,:,::-1,:]
    
    for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='F:\\Women', save_prefix='Women'+str(count), save_format='jpg',shuffle=True):
   
   
        i += 1
        if i > 10:
            break  # otherwise the generator would loop indefinitely"""
    count = count + 1        